In [1]:
import os
import sys

project_root = os.getcwd()
while "src" not in os.listdir(project_root):
    project_root = os.path.dirname(project_root)
sys.path.append(project_root)

In [ ]:
import os
import torch
import numpy as np
from typing import List, Dict, Callable
import matplotlib.pyplot as plt


# Global configuration for plotting
plt.rcParams["figure.figsize"] = [20, 6]

In [3]:
data_root = os.path.join(project_root, "data", "processed", "train")

if not os.path.exists(data_root):
    raise FileNotFoundError(
        "No se encontró la carpeta data, por favor ejecute el script download_data.sh antes de ejecutar este script."
    )

In [ ]:
from src.utils.audio import AudioSample
from src.utils.data.dataset import AudioDataset
from src.utils.data.transforms import TransformChain, FreqSplit, STFTTransform

tc = TransformChain()
convert_to_tensor: Callable[[AudioSample], Dict[str, torch.Tensor]] = (
    lambda x: x.to_tensor()
)
tc.add_transform(convert_to_tensor)
tc.add_transform(STFTTransform())

samples: List[AudioSample] = []
for file in os.listdir(data_root):
    npz_file = os.path.join(data_root, file)
    data = np.load(npz_file, allow_pickle=True)
    for key in data.files:
        samples.append(AudioSample(data[key].item()))
    break

In [ ]:
from sklearn.model_selection import train_test_split

train_samples, val_samples = train_test_split(samples, test_size=0.3)

train_dataset = AudioDataset(train_samples, tc)
val_dataset = AudioDataset(val_samples, tc)

In [6]:
from src.utils.training.loss import sdr_loss, si_sdr_loss, snr_loss
from src.utils.training.trainer import Trainer
from src.models import SimpleUNet

model = SimpleUNet(input_channels=1, output_channels=4, depth=1)

In [7]:
trainer = Trainer(
    target_keys=["drums", "bass", "vocals", "other"],
    model=model,
    loss_fn=si_sdr_loss,
    optimizer=torch.optim.Adam(model.parameters(), lr=0.001),
)

trainer.train(train_dataset, epochs=20, batch_size=4)

2025-02-10 02:48:20,223 - INFO - Starting training for 50 epochs.
2025-02-10 02:48:29,119 - INFO - Epoch 1/50 - Loss: 36.7422      


Epoch 1/50 - Loss: 36.7422


2025-02-10 02:48:37,621 - INFO - Epoch 2/50 - Loss: 25.3931      


Epoch 2/50 - Loss: 25.3931


2025-02-10 02:48:46,182 - INFO - Epoch 3/50 - Loss: 17.4122      


...

Epoch 48/50 - Loss: 0.8570


2025-02-10 02:55:20,046 - INFO - Epoch 49/50 - Loss: 0.8704      


Epoch 49/50 - Loss: 0.8704


2025-02-10 02:55:28,406 - INFO - Epoch 50/50 - Loss: 0.8297      


Epoch 50/50 - Loss: 0.8297


In [ ]:
trainer.validate(val_dataset, batch_size=4)

In [ ]:
model_path = os.path.join(project_root, "experiments/results", "simple_unet.pth")

# Save the model
torch.save(model.state_dict(), model_path)

RuntimeError: Parent directory /home/nhrot/Programming/Python/DeepLearning/DeepSampler/models does not exist.